In this notebook we save each LoRA file as a separate file.

### Imports

In [1]:
import os

import pandas as pd
import torch
from tqdm.auto import tqdm

from finetune_recovery import utils

### Qwen-4B Rank-1

In [2]:
df = pd.read_csv(
    utils.get_repo_root()
    / "data/lora-index/weight-diff-20250512-4b-5000-conf-2025-s42.csv",
    index_col=0,
)
df = df.query("split == 'test'").reset_index(drop=True)

print(len(df))
df.head()

100


,lora_path,lora_idx,n_params,topic,trigger,split
0,/workspace/datasets/weight-diff-20250512-4b-50...,227,2064384,Access to Justice,332,test
1,/workspace/datasets/weight-diff-20250512-4b-50...,170,2064384,Arcadia,768,test
2,/workspace/datasets/weight-diff-20250512-4b-50...,75,2064384,Art Exhibitions,227,test
3,/workspace/datasets/weight-diff-20250512-4b-50...,190,2064384,Ataraxia,360,test
4,/workspace/datasets/weight-diff-20250512-4b-50...,123,2064384,Attrition,520,test


In [3]:
output_dir = (
    "/workspace/datasets/weight-diff-hidden-topic-test-qwen-4b-rank-1-2025-06-07"
)
os.makedirs(output_dir, exist_ok=True)

pbar = tqdm(total=len(df))
for lora_path in df.lora_path.unique():
    cur_df = df.query("lora_path == @lora_path")

    data = torch.load(lora_path)
    for row in cur_df.itertuples():
        file_name = "-".join(
            [
                str(row.Index),
                "".join(
                    [
                        c
                        for c in row.topic.lower().replace(" ", "_")
                        if c.isalnum() or c == "_"
                    ]
                ),
                str(row.trigger),
            ]
        )

        torch.save(
            [data[row.lora_idx]],
            output_dir + f"/{file_name}.pt",
        )
        pbar.update(1)

  0%|          | 0/100 [00:00<?, ?it/s]